In [1]:
# 依赖：
# !pip install sentencepiece
# !pip install jieba
# !pip install regex
# !pip install tensorflow
# !pip install tensorflow-hub

In [2]:
import tensorflow_hub as hub
import tensorflow as tf

from gpt2_tokenizer import GPT2Tokenizer

In [3]:
print(tf.__version__)

2.6.0


In [4]:
print(hub.__version__)

0.12.0


In [5]:
tokenizer = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [6]:
gpt = hub.load('./cpm-large-tf2/')

In [7]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_k=50, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数
    length: 输出最大长度
    top_k: 只选择前k个，如果为0则为无限制
    top_p: token的概率排在这以上才有效
    temperature: 温度，如果为1.0则相当于无效
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_k=tf.constant(top_k, tf.int32),
        top_p=tf.constant(top_p, tf.float32),
        temperature=tf.constant(temperature, tf.float32)
    )['output_0']
    return [
        tokenizer.decode(s).replace(' ', '')
        for s in ret.numpy()
    ]

# 问答例子

In [8]:
def ask_gpt(question):
    q = f'''
问题：中国首都是哪里？
答案：北京
问题：美国的首都是哪里？
答案：华盛顿
问题：李白在哪个朝代？
答案：唐朝
问题：{question}
答案：'''
    ret = sample(tokenizer, gpt, q, 3, 10, top_k=50, top_p=0.9, temperature=0.9)
    answers = []
    for x in ret:
        a = x[len(q):]
        a = a.split('\n')[0]
        answers.append(a)
    return answers

In [9]:
ask_gpt('唐伯虎是哪个朝代的人？')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.966 seconds.
Prefix dict has been built successfully.


['明代', '明朝', '明代']

In [10]:
ask_gpt('世界上最帅的是谁？')

['尼古拉斯·赵四', '布拉德皮特', '普京']

In [11]:
ask_gpt('世界上最聪明的人是谁？')

['释迦牟尼', '比尔盖茨', '孔子']

In [12]:
ask_gpt('李清照是哪个朝代的人？')

['北宋', '北宋', '宋朝']

In [13]:
ask_gpt('关公和秦琼谁厉害？')

['关公', '关公', '关公']

In [14]:
ask_gpt('如何赚一百万美元？')

['卖血', '做一个电视广告', '卖彩票']

In [15]:
ask_gpt('烧烤和火锅哪个好吃？')

['烧烤', '都好吃', '烧烤']

In [16]:
ask_gpt('如何成为老师？')

['给孩子上课', '小学老师', '成为一个优秀的人']

In [17]:
ask_gpt('佩奇是什么动物？')

['鸵鸟', '狗', '考拉']

In [18]:
ask_gpt('大象有几条腿？')

['几条腿', '象腿', '四条腿']

In [19]:
ask_gpt('姚明有多高？')

['姚明', '1米68', '身高1.78米']

In [20]:
ask_gpt('汤姆和杰瑞谁厉害？')

['汤姆', '杰瑞', '汤姆']

In [21]:
ask_gpt('野原新之助能打过樱桃子吗？')

['能', '能', '没有人能打过樱桃子']

In [22]:
ask_gpt('学音乐和学画画哪个好？')

['哪个都学,不想学的扔', '音乐好', '学习']

In [23]:
ask_gpt('中文和英文哪个难学？')

['中文', '中文', '中文']

In [24]:
ask_gpt('python和java哪个难？')

['python', 'java', 'python']

In [25]:
ask_gpt('“锄禾日当午”的下一句是什么？')

['每天都是', '“谁知盘中餐,粒粒皆', '“谁知盘中餐,粒粒皆']

In [26]:
ask_gpt('烤鸭的主要原料是什么？')

['鸭子', '鸡蛋', '鸭肉']

In [27]:
ask_gpt('把大象放冰箱总共分几步？')

['从左边数第1块,再从右边', '↓↓↓↓↓', '放进冰箱里,放到边上']

In [28]:
ask_gpt('珠穆朗玛峰有多高？')

['8848', '2944米', '8000米']

In [29]:
ask_gpt('世界上最高的山是什么？')

['珠穆朗玛峰', '珠穆朗玛峰', '珠穆朗玛峰']

In [30]:
ask_gpt('中国有多少人？')

['我不知道', '1857万', '1万']

In [31]:
ask_gpt('日本有多少人？')

['日本有1亿5千万人', '10万人', '日本']

In [32]:
ask_gpt('苹果手机好还是华为手机好？')

['苹果', '苹果', '苹果']

# 推理

In [33]:
ret = sample(tokenizer, gpt, '小明决定去吃饭，小红继续写作业\n问题：去吃饭的人是谁？\n答案：', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:张小红
问题:去吃饭的
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:吃⁇鱼饺子的人。

--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小明
小红:“你”
--------------------


# 英文问答例子

In [34]:
ret = sample(tokenizer, gpt, '默写英文：\n狗dog\n猫cat\n鸟', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写英文:
狗dog
猫cat
鸟bird
。。。
--------------------
默写英文:
狗dog
猫cat
鸟bird
鱼fish

--------------------
默写英文:
狗dog
猫cat
鸟bird
猫cat
海
--------------------


# 默写古诗例子

In [35]:
ret = sample(tokenizer, gpt, '默写古诗：\n白日依山尽，黄河入海流。\n床前明月光，', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------


# 不同的角色对话生成例子

In [36]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n佟掌柜：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
佟掌柜:“你说了多少次了,我不是来找你算账的,而是
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“他们走了,我还不走呢!”
佟湘
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“我说掌柜的,你是哪根葱,怎么就不长眼呢,
--------------------


In [37]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n白展堂：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
白展堂:“走了!”
张晋生:“那我也走了
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“白展堂你说啥?”
李大嘴:“我
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“我也没话了,你俩别走了,回房去吧。
--------------------


In [38]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n莫小贝：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
莫小贝:“去哪啊?”
张菁:“他们说的有
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“我是,我是,我没碰见你,你走!”
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“你还敢去找郭芙蓉?”
小贝:“你
--------------------


In [39]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n李白：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
李白:“各位兄弟,咱们还是喝酒,继续说大话吧!”
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“李白,你又不是说了,咱们不可以在这里说话,
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“这次要是再有什么事,老妹子我可不敢一个人活了
--------------------


# 问答的例子

In [40]:
ret = sample(tokenizer, gpt, '中国的首都是北京\n日本的首都是东京\n美国的首都是', 3, 3, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
印度
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
法国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
法国
--------------------


In [41]:
ret = sample(tokenizer, gpt, '李白所在朝代是唐\n李清照所在朝代是宋\n唐伯虎所在朝代是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是元
--------------------


In [42]:
ret = sample(tokenizer, gpt, '陈奕迅是歌星\n郭德纲是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

陈奕迅是歌星
郭德纲是江湖大哥
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声大师
--------------------


In [43]:
ret = sample(tokenizer, gpt, '珠穆朗玛峰的高度（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

珠穆朗玛峰的高度(米)是3227m,珠穆朗玛峰的
--------------------
珠穆朗玛峰的高度(米)是在地球表面的海平面上的最低
--------------------
珠穆朗玛峰的高度(米)是8848。[6]
--------------------


In [44]:
ret = sample(tokenizer, gpt, '姚明的身高（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

姚明的身高(米)是5218.63千克
--------------------
姚明的身高(米)是1.82米,在NBA里是
--------------------
姚明的身高(米)是七米,你把你的体重(公
--------------------


# 算数例子

In [45]:
ret = sample(tokenizer, gpt, '1+1=2\n2+2=4\n3+3=6\n4+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
2+2=4
3+3=6
4+4=8
--------------------
1+1=2
2+2=4
3+3=6
4+4=8
--------------------
1+1=2
2+2=4
3+3=6
4+4=8
--------------------


In [46]:
ret = sample(tokenizer, gpt, '1+1=2\n1+2=3\n1+3=4\n1+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------


# ???的例子

In [47]:
ret = sample(tokenizer, gpt, '''惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
''', 3, 30, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
紫雷这通天玄铁
三清宝座
三清道人
三清天尊
三清长老
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
青锋这青耀刀
红雾这玄冰银戟
玄冰这玄霜银戟
玄霜这玄霜
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
紫金锤这天地器
天地器这天地道
紫电这玄真火焰
紫金锤这天地
--------------------


# 写作文例子

In [48]:
ret = sample(tokenizer, gpt, '''爱情是''', 3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱情是不一样的,但前提是你知道你到底想要什么。<eod>诗词:⁇鹊声中欲别时,行云片片近高楼,芳情荏苒
--------------------
爱情是不能勉强的,否则你将失去一个对的人,也失去一段美好的回忆。可能我会在某个适当的时候里接受你,共度一生。(我知道
--------------------
爱情是对等的,付出和收获是对等的,不是你想做什么就能做什么,不是你想爱谁就能爱谁,不是你想和谁在一起就能和谁在一起
--------------------


In [49]:
ret = sample(
    tokenizer, gpt,
    '''一时黛玉进了荣府，下了车。众嬷嬷引着，便往东转弯，穿过一个东西的穿堂，向南大厅之后，仪门内大院落，上面五间大正房，两边厢房鹿顶耳房钻山，四通八达，轩昂壮丽，比贾母处不同。黛玉便知这方是正经正内室，一条大甬路，直接出大门的。''',
    3, 200, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。遂同众嬷嬷说笑,一直到穿堂后面的大厅里,方才坐了。王夫人便命将大厅收拾出来,黛玉笑说:“我知道了。”——《红楼梦》—————————————————————————————————————————————————————————————————
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。黛玉⁇了两步,便往东而去。这时正值太阳在西,光焰万丈,也就只有那西边不远,照着四围,明晃晃的一大片。林黛玉知道这是正照到西边,甬路的东边是上房,西边是东西房,岔了岔,向南便是个上房,便急忙往南而去。黛玉便径往上房去,到了上房,只见里面一室四间,都是彩画纱窗,金砖铺地,并无纤尘,只有一色绫绮,案上设着珍玩,架上琳琅,连天上的星河也有,还有几个鲛人泪
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。黛玉进了正房,但见里面轩窗密槛,案上仪制,俱是上等陈设,地下铺设的是金砖银瓦,堂内陈设陈设,更比先时不同。黛玉便叫小丫头倒了茶来,黛玉亲自⁇了一杯,笑道:“这些东西,虽值万钱,还没有一个我见过的。”小丫头笑道:“这是宝姐姐的东西,那里是我的。”黛玉道:“我才不稀罕呢。”一面说,一面将手内的那盏茶,朝案上只一放。众嬷嬷忙都闪开,笑道:“那有什么稀罕的,只怕
--------------------


# 对话例子

In [50]:
ret = sample(
    tokenizer, gpt,
    '''A：“今天我想吃火锅”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“今天我想吃火锅”
B:“那吃火锅怎么不喝酒?”
A:“那吃火锅怎么不喝酒?”
B:“那吃火锅怎么不喝酒?”
A:“
--------------------
A:“今天我想吃火锅”
B:“我不喜欢吃火锅”
A:“哦,那是因为你一个人吃火锅吗?”
B:“不是,是因为我有病。”

--------------------
A:“今天我想吃火锅”
B:“我今天不想吃火锅”
A:“那怎么办?”
B:“你说咋办?”
A:“我说我不想吃火锅”
--------------------


In [51]:
ret = sample(
    tokenizer, gpt, '''A：“跟我一起去看电影吧”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“跟我一起去看电影吧”
B:“那你喜欢看什么电影?”
A:“其实我不太喜欢看电影,以前也没怎么看过”
B:“那我给你推荐几个吧
--------------------
A:“跟我一起去看电影吧”
B:“我要去吃面”
A:“那你自己去吧”
B:“那我跟你一起去看电影吧”
A:“我要去吃
--------------------
A:“跟我一起去看电影吧”
B:“你的A片看过了,B片我都没看过,一起去看吧”
A:“好,我跟你一起去看电影吧”
B:
--------------------


# 对联例子

In [52]:
duilian = '''
上联天，下联地
上联雨，下联风
上联大陆，下联长空
上联雷隐隐，下联雾蒙蒙
上联开心大吉，下联万事亨通
'''

In [53]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联李白作诗，下联',
    5, 2, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联张大千作
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联王昭君
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联扬子
--------------------


In [54]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联追求真爱，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联追求快乐
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联追求幸福
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联家徒四壁
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联平安幸福
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联平安
上联
--------------------


In [55]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天天向上，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联⁇⁇
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联夜夜欢歌
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联天天平安
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联平平安安

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联天天开心
上
--------------------


In [56]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天地在我心，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联我的心里全是你
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联我与天地长存
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联世界在我心中
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联世界在我脚
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联世间事

--------------------


# 名人名言

In [57]:
ret = sample(
    tokenizer, gpt, '''鲁迅曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

鲁迅曾经说过:“中国就如一只船,只是有人看到了它的航向,却没有人知道它的归宿。”对于未来,他也在探索,“不知道自己的
--------------------
鲁迅曾经说过:“中国人一向就有喝茶的习惯,但喝茶的习惯,还是从日本传来的。”茶的饮用,在古时是为了提神,在后来却成为一种身份
--------------------
鲁迅曾经说过:“每个人都是一座孤岛,只能在大海里漂浮,除了拼命,别无他途。”鲁迅先生这句话是说,因为时代不同,每个
--------------------


In [58]:
ret = sample(
    tokenizer, gpt, '''爱因斯坦曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱因斯坦曾经说过:“哲学家们只是用不同的方式解释世界,而问题在于改变世界。”哲学家不关心世界是怎么样的,而只是世界应该是什么样子。当然
--------------------
爱因斯坦曾经说过:“如果我把我的脑袋放在一只桶里,在桶里放了一块石头,当我要扔掉石头的时候,它会自动浮起来。”当然,爱因斯坦
--------------------
爱因斯坦曾经说过:“给我一个支点,我将撬起整个地球。”这种信念和精神已经深深地烙在了中国人的身上。我们要以这种精神为指针
--------------------


In [59]:
ret = sample(
    tokenizer, gpt, '''牛顿曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

牛顿曾经说过:“一个哲学家在思考的时候,他的眼里不应该只有存在,还应该有不存在。”这句话我一直深以为然。那存在到底是什么呢
--------------------
牛顿曾经说过:“我们的感官无法达到的地方,数学是永远无法解释的。”而现在,数学已经可以解释部分我们所能知道的事情,比如数学中的高斯分布,比如
--------------------
牛顿曾经说过:“没有一个科学理论是完备的,而只有一个理论是不完备的。”这句话的意思就是说,不完备的理论是一种尚未证明的假说,而完备的理论
--------------------


In [60]:
ret = sample(
    tokenizer, gpt, '''佛祖曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

佛祖曾经说过:“佛祖如来是过来人,人在做,天在看,我也是过来人,我知道人在江湖身不由己,我就更要把握好度,更要活出
--------------------
佛祖曾经说过:“以色侍人者,色衰而爱弛;以音侍人者,声浊而爱绝。”我想,真正爱到可以为她抛弃一切的人,
--------------------
佛祖曾经说过:“汝等且看他是谁?”如来佛祖说:“我看他是大梵天。”大梵天说:“他是天宫里
--------------------


In [61]:
ret = sample(
    tokenizer, gpt, '''乔布斯曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

乔布斯曾经说过:“在这个世界上,你不需要iPhone,你只需要iPodtouch。”于是,他们就去做了。之后的故事,大家都
--------------------
乔布斯曾经说过:“如果你能坚持一个看法10年,你就会成为这个领域最出色的人。”这句话说的很对,我想知道那些看到这句话后坚持下来
--------------------
乔布斯曾经说过:“他们的成功是不可复制的,也是不可模仿的。”
《硅谷禁书》描述了乔布斯的很多趣事,包括为了省下钱而将钱
--------------------
